# Simulación de información de distribuidores

**Grupo 8 - Integrantes:**
*   **Santino Semec**
*   **Gerardo Toboso**
*   **Agustín Rebechi**
*   **Nicolás Hurtado**
*   **Bruno Sidelsky**

Este notebook se encargará de simular la generación de datos por parte de los distribuidores, todo con el objetivo de recolectar la información 
suficiente para poder responder preguntas de negocio las cuales serán de gran utilidad para **Argentina Ideal**.

# Setup y lectura de archivos

Como primer paso importamos las librerías a utilizar:

In [1]:
import csv
import numpy as np
import pandas as pd
import string as st
from datetime import datetime
from datetime import timedelta
from geopy.geocoders import ArcGIS
import random
import os 

Guardamos las rutas con las cuales estaremos trabajando, estas mismas se usarán para poder generar
los archivos correctamente:

In [2]:
directorio_actual = os.getcwd()

#Me guardo el path de las especificaciones
path = os.path.join(directorio_actual,'TP_Final_Especificacion_1.xlsx')

#A este path se guardarán los CSV una vez generados
path_destino = directorio_actual

Finalmente leemos **el archivo de especificaciones** el cual contiene información de como se generará la información de cada 
tabla generada por los distribuidores:

In [3]:
#Leo los archivos de especificiaciones
df_sheets_desc = pd.read_excel(path, sheet_name=None)
df_cond_vta = df_sheets_desc['Condicion_Venta']
df_unidades = df_sheets_desc['Unidades']
df_tipo_negocio = df_sheets_desc['Tipo_Negocio']
df_cadena = df_sheets_desc['Cadena']
df_dias_visita = df_sheets_desc['Dias_Visita']
df_productos = df_sheets_desc['Productos']
df_cond_venta = df_sheets_desc['Condicion_Venta']
df_prov_localidades = df_sheets_desc['Prov_Localidades']
df_estado = df_sheets_desc['Estado_cliente']
df_dias_visita = df_sheets_desc['Dias_Visita']

# Definición de funciones útiles

In [4]:
def crear_directorio(ruta_archivos: str, nombre_directorio: str) -> str:
    """
    Crea un directorio en la ruta especificada si no existe y devuelve la ruta del directorio creado o existente.

    Args:
        ruta_archivos (str): La ruta donde se creará el directorio.
        nombre_directorio (str): El nombre del directorio a crear.

    Returns:
        str: La ruta completa del directorio creado o existente.
    """
    #Combino la ruta pasada como argumento con el nombre del directorio para obtener la ruta completa del nuevo directorio
    nueva_ruta = os.path.join(ruta_archivos, nombre_directorio)

    #Verifico si el directorio ya existe
    if not os.path.exists(nueva_ruta):
        try:
            os.mkdir(nueva_ruta)
        except OSError as e:
            print(f"No se pudo crear el directorio '{nombre_directorio}' en {ruta_archivos}: {e}")

    return nueva_ruta

In [5]:
def generar_codigos_sku(productos:list) -> dict:
    """
    Esta función genera un conjunto de códigos SKU únicos para una lista de productos.

    Args:
        productos (list): Una lista de productos para los cuales se deben generar códigos SKU.

    Returns:
        dict: Un diccionario que contiene los nombres de los productos como claves y los códigos SKU como valores.
    """
    codigos_sku = {}
    for producto in productos:
        #Defino el conjunto de caracteres 
        caracteres = st.ascii_uppercase + st.digits
        
        #Genero un código SKU aleatorio de 10 caracteres usando el conjunto de caracteres definido
        sku_codigo = ''.join(random.choice(caracteres) for _ in range(10))
        
        #Agrego el código SKU al diccionario como clave del producto al que corresponde
        codigos_sku[producto] = sku_codigo

    return codigos_sku

In [6]:
def to_string_date(our_date) -> str:
    """
    Esta función toma un objeto datetime y devuelve una cadena en el formato YYYY-MM-DD

    Args:
        our_date: un objeto datetime

    Returns:
        str: una cadena en el formato YYYY-MM-DD
    """
    our_date_string = f'{our_date.year}-{our_date.month:02d}-{our_date.day:02d}'
    return(our_date_string)

In [7]:
def get_date_random_str(start_date: str, end_date: str) -> str:
    """
    Esta función toma dos cadenas en el formato "YYYY-MM-DD" y devuelve una fecha aleatoria dentro del rango especificado.

    Args:
        start_date (str): la fecha de inicio del rango
        end_date (str): la fecha de finalización del rango

    Returns:
        str: una fecha aleatoria dentro del rango especificado, formateada como "YYYY-MM-DD"
    """
    start_year = int(start_date[0:4])
    start_month = int(start_date[6:7])
    start_day = int(start_date[8:10])
    end_year = int(end_date[0:4])
    end_month = int(end_date[6:7])
    end_day = int(end_date[8:10])

    start = datetime(start_year, start_month, start_day)
    end =  datetime(end_year, end_month, end_day)
    random_date = start + (end - start) * random.random()
    return(to_string_date(random_date))


In [8]:
def get_random_lat_long(country: str, city: str) -> float:
    """
    Esta función toma un país y una ciudad como entrada y devuelve 
    una coordenada aleatoria de latitud y longitud para una ubicación dentro de ese país.

    Args:
        country (str): el nombre del país
        city (str): el nombre de la ciudad

    Returns:
        float: una coordenada aleatoria de latitud
        float: una coordenada aleatoria de longitud

    Raises:
        ValueError: si la entrada del país o la ciudad no se reconoce
    """
    geolocator = ArcGIS()
    location = geolocator.geocode(f"{city}, {country}")

    if location:
        lat = location.latitude
        long = location.longitude
        # Añado un poco de aleatoriedad a las coordenadas
        lat += random.uniform(-0.05, 0.05)
        long += random.uniform(-0.05, 0.05)
        return lat, long
    else:
        raise ValueError(f"No se pudo encontrar coordenadas para {city}, {country}.")

In [9]:
def obtener_locaciones_aleatorias(df_localidades: pd.DataFrame, cantidad_localidades: int) -> list:
    """
    Genera ubicaciones aleatorias a partir de un dataframe que contiene provincias y localidades.

    Args:
        df_localidades (pd.DataFrame): DataFrame que contiene las provincias de Argentina como columnas y las localidades de cada provincia como filas.
        cantidad_localidades (int): Número de ubicaciones a generar.

    Returns:
        tuple: Una tupla de listas con la siguiente información:
            - Lista de provincias aleatorias.
            - Lista de ciudades aleatorias correspondientes a las provincias.
            - Lista de latitudes pertenecientes a las ubicaciones (Provincia, Ciudad).
            - Lista de longitudes pertenecientes a las ubicaciones (Provincia, Ciudad).

    Note:
        La información resultante está ordenada según el orden interno de las 4 listas.
    """
    #Obtengo las provincias random
    provincias = [(random.choice(df_localidades.columns.values)) for _ in range(cantidad_localidades)]

    #Genero una lista donde se guardarán las ciudades aleatorias para cada provincia
    ciudades_aleatorias = []

    #Recorro las provincias random que obtuve
    for p in provincias:
        #Selecciono las localidades de la provincia p
        ciudades_provincia = df_localidades[p].dropna().tolist()

        #Para cada provincia en la lista tomo una localidad aleatoria
        ciudad_aleatoria = random.choice(ciudades_provincia)

        #Meto la localidad obtenida en la lista
        ciudades_aleatorias.append(ciudad_aleatoria)

    #Genero 2 listas para cada coordenada
    coordenada_latitud = []
    coordenada_longitud = []

    #Para cada cliente debo generar un par de coordenadas
    for _ in range(cantidad_localidades):
        # Uso la función definida anteriormente para la generación de las localizaciones
        latitud, longitud = get_random_lat_long(provincias[_], ciudades_aleatorias[_])
        coordenada_latitud.append(latitud)
        coordenada_longitud.append(longitud)

    return provincias, ciudades_aleatorias, coordenada_latitud, coordenada_longitud

In [10]:
def generar_clientes(cant_clientes:int, df_localidades: pd.DataFrame, df_estado_cliente: pd.DataFrame, df_negocio_tipo: pd.DataFrame) :
    '''
    Genera información diaria particular de cada cliente.

    Esta función genera información para una cantidad específica de clientes, incluyendo nombres únicos,
    tipos de negocio, números de teléfono, direcciones, razones sociales, CUIT, estados, provincias,
    ciudades aleatorias, coordenadas de latitud y longitud.

    Args:
        cant_clientes (int): La cantidad de clientes que se desean generar.
        df_localidades (pd.DataFrame): Dataframe que contiene información sobre las localidades.
        df_estado_cliente (pd.DataFrame): Dataframe que contiene información sobre los estados de los clientes.
        df_negocio_tipo (pd.DataFrame): Dataframe que contiene información sobre los tipos de negocio.

    Returns:
        tuple: Una tupla con las siguientes listas en orden:
            - nombres_clientes (list): Nombres únicos para cada cliente.
            - tipos_negocio (list): Tipos de negocio para cada cliente.
            - telefonos (list): Números de teléfono únicos para cada cliente.
            - direcciones (list): Direcciones únicas para cada cliente.
            - razones_sociales (list): Razones sociales únicas para cada empresa.
            - cuit (list): CUIT únicos para cada cliente.
            - estados (list): Estados para cada cliente.
            - provincias (list): Provincias para cada cliente.
            - ciudades_aleatorias (list): Ciudades aleatorias para cada cliente.
            - coordenada_latitud (list): Coordenadas de latitud para cada cliente.
            - coordenada_longitud (list): Coordenadas de longitud para cada cliente.
    '''
    
    #Obtengo los datos de ubicaciones de cada cliente
    provincias, ciudades_aleatorias, coordenada_latitud, coordenada_longitud = obtener_locaciones_aleatorias(df_localidades=df_localidades,
                                                                                                             cantidad_localidades=cant_clientes)
    
    #Seleccionamos un estado para cada cliente, ver qué tiene más probabilidad de ser activo que inactivo
    estados = [random.choices(df_estado_cliente['nombre'], weights= df_estado_cliente['probabilidades'], k=1)[0] for _ in range(cant_clientes)]
    
    #Genero nombres únicos para cada cliente
    nombres_clientes = [f'Cliente_{i+1}' for i in range(cant_clientes)]
    
    #Genero razones sociales únicas para cada empresa
    razones_sociales = [(f"Empresa_{i+1}") for i in range(cant_clientes)]
    
    #Genero direcciones únicas para cada cliente
    direcciones = [(f"Dirección_{i+1}") for i in range(cant_clientes)]
    
    #Genero tipos de negocio para cada cliente
    tipos_negocio = [random.choice(df_negocio_tipo['nombre']) for _ in range(cant_clientes)]
    
    #Genero CUIT únicos para cada cliente
    cuit = set()
    while len(cuit) < cant_clientes:
        cuit.add(random.randint(10000000, 99999999))
    cuit = list(cuit)
        
    #Genero números de teléfono únicos para cada cliente
    telefonos = set()
    while len(telefonos) < cant_clientes:
        telefonos.add(f"{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}")
    telefonos = list(telefonos)
    
    return nombres_clientes, tipos_negocio, telefonos, direcciones, razones_sociales, cuit, estados, provincias, ciudades_aleatorias, coordenada_latitud, coordenada_longitud


In [11]:
def generar_codigo_sucursal(cant_sucursales:int, n_distribuidor:int):
    '''
    Función que se encarga de generar los codigos de las sucursales del distribuidor, 
    según la cantidad de sucursales que se necesiten.
    '''
    
    #Defino un número el cual será la base para el rango de valores que toman los códigos
    #De sucursales de los distribuidores
    codigo_sucursal_base = (n_distribuidor * 100)  

    #A partir del número base genero los códigos dentro de un rango de 100 números
    codigos_sucursal_def = set()
    while len(codigos_sucursal_def) < cant_sucursales:
        codigos_sucursal_def.add(codigo_sucursal_base + random.randint(10, 99))
    codigos_sucursal_def = list(codigos_sucursal_def)
    
    return codigos_sucursal_def, codigo_sucursal_base

In [12]:
def asociar_sucursal_cliente(lista_sucursales:list, cant_clientes:int, fecha_cierre:datetime,codigo_sucursal_base:int, df_dias:pd.DataFrame):
    """
    Asocia una sucursal a cada cliente para un día específico.

    Args:
        lista_sucursales (list): Lista de sucursales disponibles.
        cant_clientes (int): Cantidad de clientes.
        fecha_cierre (datetime): Fecha para la asociación.
        codigo_sucursal_base (int): Número base para el código de las sucursales.
        df_dias (pd.DataFrame): DataFrame con los días de visita.

    Returns:
        tuple: Una tupla de listas con la siguiente información:
            - Lista de códigos de clientes.
            - Lista de sucursales asociadas a cada cliente.
            - Lista de fechas de cierre (misma fecha para todos los clientes).
            - Lista de días de visita de los distribuidores a los minoristas.

    Note:
        La información resultante está ordenada según el orden interno de las listas.
    """
    
    #Genero una lista de fechas según la fecha dada, esto con el propósito de que cada relación tenga
    #una fecha asignada
    fecha_cierre_string = f'{fecha_cierre.year}-{fecha_cierre.month:02d}-{fecha_cierre.day:02d}'
    fecha_cie_com_data = [fecha_cierre_string for _ in range(cant_clientes)]
    
    #Uso el número base de la sucursal para generar el código de los clientes
    codigos_clientes = set()
    while len(codigos_clientes) < cant_clientes:
        codigos_clientes.add(codigo_sucursal_base + random.randint(1000, 9999))
    codigos_clientes = list(codigos_clientes)
    
    #Ahora le asigno una sucursal a cada cliente 
    sucursal_cliente = [random.choice(lista_sucursales) for _ in range(cant_clientes)]
    
    #Genero los días de visita por parte del distribuidor hacia los minoristas
    dias_visita = [(random.choice(df_dias['codigo_dia'])) for _ in range(cant_clientes)]
    
    return codigos_clientes, sucursal_cliente, fecha_cie_com_data, dias_visita

In [13]:
def generar_info_stock(productos_sku:dict, cant_clientes:int, df_unidad:pd.DataFrame):
    """
    Genera información sobre el stock de productos para un número dado de clientes.

    Args:
        productos_sku (dict): Un diccionario que mapea nombres de productos a códigos SKU.
        cant_clientes (int): El número de clientes para los que se generará información de stock.
        df_unidad (pd.DataFrame): Un DataFrame que contiene información sobre las unidades de los productos.

    Returns:
        tuple: Una tupla que contiene:
            - Una lista de códigos SKU asignados a cada cliente.
            - Una lista de productos comprados por cada cliente.
            - Una lista de cantidades de stock para cada cliente.
            - Una lista de unidades correspondientes al stock de cada cliente.
    """
    
    #Como a cada cliente se le vendió un producto, le asigno un código SKU a cada cliente
    #según el producto que compró
    codigos_sku = [random.choice(list(productos_sku.values())) for _ in range(cant_clientes)]
    
    #Ahora a cada código sku le asigno su producto correspondiente
    productos = []
    
    #Invierto el diccionario para facilitar la búsqueda
    diccionario_invertido = {valor: clave for clave, valor in productos_sku.items()}
    
    for codigo_sku in codigos_sku:
        
        productos.append(diccionario_invertido[codigo_sku])
        
    #Por último genero la información sobre los productos en stock de forma aleatoria
    unidades = [(random.choice(df_unidad['codigo_unidad'])) for _ in range(cant_clientes)]
    stock = [(random.randint(100, 500) if unidades[_] == "UNI" else random.randint(1, 100)) for _ in range(cant_clientes)]

    return codigos_sku, productos, stock, unidades

# Generación de archivos

In [14]:
#Genero los SKUs correspondientes para cada producto
codigos_sku = generar_codigos_sku(df_productos['nombre'])

#Obtengo los nombres de las columnas de cada especificación
columns_v = df_sheets_desc['Desc_Ventas']['Campo']
columns_s = df_sheets_desc['Desc_Stock']['Campo']
columns_m = df_sheets_desc['Desc_Clientes']['Campo'] #La m es de maestro!!!
columns_d = df_sheets_desc['Desc_Deuda']['Campo']


#Cantidad de días desde la fecha actual hacia el pasado que generamos datos
cant_dias = 2

#Fecha actual
fecha_actual = datetime.now()

#Cantidad de distribuidores
cant_dist = 2

#Cantidad de clientes
cant_clientes = 20

#Cantidad sucursales por distribuidor
cant_sucursales = 3

#Para cada distribuidor generamos sus respectivos archivos
for distribuidor in range(1,cant_dist+1):

  #Cada CSV corresponde a un día
  for d in range(0,cant_dias):
    
  
    #------------------------------------------------
    #INFORMACIÓN PARA ARCHIVO MAESTRO (CLIENTES) 
    
    #Genero información asociada a cada cliente
    nombre_cliente, tipo_negocio, telefono, direccion, razon_social, cuit, estado, provincias, ciudades_aleatorias, coordenada_latitud, coordenada_longitud = generar_clientes(cant_clientes=cant_clientes, 
                                                                                                                                                                               df_localidades=df_prov_localidades, 
                                                                                                                                                                               df_estado_cliente=df_estado, 
                                                                                                                                                                               df_negocio_tipo= df_tipo_negocio)
    #Genero información sobre la fecha de cierre para cada cliente del distribuidor
    #VER que la fecha cierre en cada iteración se va reduciendo en 1 según la cantidad de días 
    #que especificamos
    fech_cierre = fecha_actual - timedelta(days=d)
    
    #Genero los códigos para las sucursales indicadas en 'cant_sucursales'
    codigos_sucursales, codigo_base_sucursales = generar_codigo_sucursal(cant_sucursales= cant_sucursales, n_distribuidor= distribuidor)
    
    #A cada cliente le asigno un código según la sucursal con la que hizo trato y además obtengo la fecha de cierre comercial 
    #formateada
    clientes, sucursales_dist, fech_cie_com_data, dias_visita = asociar_sucursal_cliente(lista_sucursales= codigos_sucursales, cant_clientes= cant_clientes, 
                                                                            fecha_cierre= fech_cierre, codigo_sucursal_base= codigo_base_sucursales, df_dias= df_dias_visita)  
      
    #Fecha de alta y de baja
    fecha_alta = [get_date_random_str('2024-03-01', '2024-05-31') for _ in range(cant_clientes)]  # Fecha de alta
    fecha_baja = [get_date_random_str('2024-06-01', '2024-09-30')for _ in range(cant_clientes)]  # No hay fecha de baja
    
    #Me guardo el día formateado 
    fech_cierre_string = fech_cie_com_data[0]
    #------------------------------------------------
    
    #------------------------------------------------
    #INFORMACIÓN ARCHIVOS VENTA-STOCK

    vta_unidades_data = [(random.randint(0,100)) for _ in range(cant_clientes)]

    #Generamos datos aleatorios sobre los importes de las ventas de los clientes
    vta_importe_data = [round((random.uniform(100,1000)),2) for _ in range(cant_clientes)]

    #Para cada cliente también genero de forma aleatoria su condición de venta
    cond_vta_data = [random.choice(df_cond_vta['codigo_condicion']) for _ in range(cant_clientes)]

    
    sku_codigo, producto, stock, unidad = generar_info_stock(productos_sku= codigos_sku, cant_clientes= cant_clientes, 
                                                             df_unidad= df_unidades)
    #------------------------------------------------    

    #------------------------------------------------
    #INFORMACIÓN PARA ARCHIVO DEUDA
    
    deuda_vencida = [(round(random.uniform(0, 1000), 2)) for _ in range(cant_clientes)]
    deuda_total = [(round(random.uniform(0, 10000), 2)) for _ in range(cant_clientes)]
    #------------------------------------------------


    #------------------------------------------------
    #SUBIDA DE LOS ARCHIVOS

    #Zipeo las listas según los requerimientos de cada archivo a generar y en un cierto orden
    venta_clientes_data = list(zip(sucursales_dist,clientes,fech_cie_com_data,sku_codigo,vta_unidades_data,vta_importe_data,cond_vta_data))
    stock_data = list(zip(sucursales_dist, fech_cie_com_data, sku_codigo, producto, stock, unidad))
    clientes_data = list(zip(sucursales_dist, clientes, ciudades_aleatorias, provincias, estado, nombre_cliente, cuit, razon_social, direccion, dias_visita, telefono, fecha_alta, fecha_baja, coordenada_latitud, coordenada_longitud, cond_vta_data, deuda_vencida, tipo_negocio))
    deuda_data = list(zip(sucursales_dist,clientes, fech_cie_com_data,deuda_vencida,deuda_total ))

    #Convierto cada lista zipeada en un DataFrame
    df_vta_cli = pd.DataFrame(venta_clientes_data,columns=columns_v)
    df_stock = pd.DataFrame(stock_data,columns=columns_s)
    df_clientes = pd.DataFrame(clientes_data,columns=columns_m)
    df_deuda =  pd.DataFrame(deuda_data,columns=columns_d)

    #Me guardo una ruta destino la cual se define según el número de distribuidor en
    #el cual me encuentro iterando
    carpeta_destino =  os.path.join(path_destino, f'distribuidor_{str(distribuidor)}')
    
    #Ahora genero una carpeta con los archivos de cada día
    ruta_final = crear_directorio(ruta_archivos= carpeta_destino, nombre_directorio= f'Archivos_{fech_cierre_string}')

    #Luego guardo cada DataFrame en cada archivo destino correspondiente
    df_vta_cli.to_csv(os.path.join(ruta_final, f'venta_{fech_cierre_string}.csv'), encoding='utf-8', index=False)
    df_stock.to_csv(os.path.join(ruta_final, f'stock_{fech_cierre_string}.csv'), encoding='utf-8', index=False)
    df_clientes.to_csv(os.path.join(ruta_final, f'cliente_{fech_cierre_string}.csv'), encoding='utf-8', index=False)
    df_deuda.to_csv(os.path.join(ruta_final, f'deuda_{fech_cierre_string}.csv'), encoding='utf-8', index=False)
    #------------------------------------------------